In [1]:
import numpy as np
USE_H5PY=False
if USE_H5PY:
    import h5py
    filepath = "./refresh_test.h5"
else:
    import h5pyd as h5py
    filepath = "/home/test_user1/test/refresh_test.h5"

In [2]:
f = h5py.File(filepath, "w", libver="latest")

In [3]:
# create an extendable dataset
dset_f = f.create_dataset("dset", (3,4), maxshape = (None, None), dtype=np.int32)
dset_f

<HDF5 dataset "dset": shape (3, 4), type "<i4">

In [4]:
f.swmr_mode = True   # not strictly needed for HSDS, but compatible with h5py usage

In [5]:
g = h5py.File(filepath, swmr=True)  # read-only file handles need to set swmr mode in the constructor

In [6]:
dset_g = g["dset"]  # get a reference to the dataset via the read-only file handle
dset_g

<HDF5 dataset "dset": shape (3, 4), type "<i4">

In [7]:
dset_g[:,:]  # all zeros

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)

In [8]:
# write some values to the dataset
for i in range(3):
    dset_f[i,:] = [i, i*2, i*4, i*8]
dset_f[:,:]

array([[ 0,  0,  0,  0],
       [ 1,  2,  4,  8],
       [ 2,  4,  8, 16]], dtype=int32)

In [9]:
# the read-only handle will show the latest values
# (though with h5py swmr, you might need to do a refresh to see the changes)
dset_g[:,:] 

array([[ 0,  0,  0,  0],
       [ 1,  2,  4,  8],
       [ 2,  4,  8, 16]], dtype=int32)

In [10]:
# refresh will work with h5py or h5pyd
dset_g.refresh()
dset_g[:, :]

array([[ 0,  0,  0,  0],
       [ 1,  2,  4,  8],
       [ 2,  4,  8, 16]], dtype=int32)

In [11]:
# Resize the dataset using the writable file handle
dset_f.resize((6, 8))  

In [12]:
# the read-only reference to the dataset doesn't "see" the shape change yet
#  for both h5py and h5pyd
dset_g  


<HDF5 dataset "dset": shape (3, 4), type "<i4">

In [13]:
dset_g.refresh()  # this will re-synch this reference to the dataset to the server's view
dset_g

<HDF5 dataset "dset": shape (6, 8), type "<i4">

In [14]:
# create an attribute in the dataset
dset_f.attrs["a1"] = "an attribute"  # create an attribute

In [15]:
# don't need a refresh to see the attribute change from the read-only handle
"a1" in dset_g.attrs

True

In [16]:
# create a sub-group
f.create_group("subgrp")

<HDF5 group "/subgrp" (0 members)>

In [17]:
"subgrp" in g  # will show up in the g reference

True

In [20]:
g.refresh()  # after a refresh, subgroup will be visible
"subgrp" in g

AttributeError: 'File' object has no attribute 'refresh'

In [21]:
# close file handles
f.close()
g.close()